In [1]:
#torch==1.7.1/transformers==3.5.1
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from save_files import load_json
from sklearn.model_selection import train_test_split

In [2]:
class BERTDataset(Dataset):
    def __init__(self, dataset, bert_tokenizer, max_len,
                 pad=True, pair=True):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([d]) for d in dataset]

    def __getitem__(self, i):
        return self.sentences[i]

    def __len__(self):
        return (len(self.sentences))

In [3]:
class SentenceEmbeder:
    PAD = True
    PAIR = False
    BATCH_SIZE = 2
    def __init__(self, max_len):
        self.MAX_LEN = max_len
        self.device = torch.device("cuda:0")
        bertmodel, vocab = get_pytorch_kobert_model()
        self.bertmodel = bertmodel.to(self.device)
        tokenizer = get_tokenizer()
        self.tokenizer = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()
    
    def encode(self, sent_list):
        data_train = BERTDataset(sent_list, self.tokenizer, self.MAX_LEN, SentenceEmbeder.PAD, SentenceEmbeder.PAIR)
        train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=SentenceEmbeder.BATCH_SIZE, num_workers=5)

        result = []
        for token_ids, valid_length, segment_ids in tqdm(train_dataloader):
            token_ids = token_ids.long().to(self.device)
            segment_ids = segment_ids.long().to(self.device)
            valid_length = valid_length.to(self.device)
            attention_mask = self.gen_attention_mask(token_ids, valid_length)
            _, pooler = self.bertmodel(token_ids, attention_mask, segment_ids)

            for elem in pooler:
                result.append(elem)
            del token_ids, segment_ids, valid_length, attention_mask, pooler
            torch.cuda.empty_cache()
        return result

In [4]:
# USE_CUDA = torch.cuda.is_available()

In [5]:
raw_data_dict = load_json('history_1.json')
#'{}␞{}'
document_list = [list_elem['document'].replace('\n', '').replace('            ', '').replace('/', '\n') for list_elem in raw_data_dict['list']]

x_train, x_test, _, _ = train_test_split(
    document_list, document_list, test_size=0.2, shuffle=True, random_state=34)

In [6]:
# 후보 키워드 추출
from sklearn.feature_extraction.text import CountVectorizer

n_gram_range = (1, 1)
stop_words = "english"

count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([document_list[0]])
#count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc])
candidates = count.get_feature_names()

print(document_list[0])
print(candidates)

제목 : 5·1경기장(五一競技場) 
 이칭 : 능라도경기장, 인민대경기장 
 분야 : 체육 
 유형 : 지명 
 시대 : 현대 
 성격 : 경기장 
 규모 : 면적 20만 7000㎡ 
 소재지 : 평양특별시 능라도 
 정의 : 평양특별시 능라도에 위치하고 있는 북한의 종합경기장. 
 내용 : 1989년 5월 1일 노동절에 준공되었다. 착공 당시에는 ‘능라도경기장’으로 불리다가 1989년 4월 중앙인민위원회 정령으로 ‘인민대경기장’이라고 명명하였으나, 이틀 뒤 중앙인민위원회 정령을 다시 발표하면서 현재의 명칭으로 개칭되었다.연면적 20만7000㎡에 15만석 규모의 주경기장과 각종 보조경기장, 3개의 축구훈련장, 실내연습장을 갖추고 있다. 제13차 세계청년학생축전의 개회 및 폐회식장으로 이용된 바 있다. 
 집필자 : 개정(1996년)서성우 
 
['15만석', '1989년', '1996년', '1경기장', '1일', '20만', '20만7000', '3개의', '4월', '5월', '7000', '五一競技場', '각종', '갖추고', '개정', '개칭되었다', '개회', '경기장', '규모', '규모의', '내용', '노동절에', '능라도', '능라도경기장', '능라도에', '다시', '당시에는', '면적', '명명하였으나', '명칭으로', '발표하면서', '보조경기장', '북한의', '분야', '불리다가', '서성우', '성격', '세계청년학생축전의', '소재지', '시대', '실내연습장을', '연면적', '위치하고', '유형', '으로', '이라고', '이용된', '이칭', '이틀', '인민대경기장', '있는', '있다', '정령으로', '정령을', '정의', '제13차', '제목', '종합경기장', '주경기장과', '준공되었다', '중앙인민위원회', '지명', '집필자', '착공', '체육', '축구훈련장', '평양특별시', '폐회식장으로', '현대', '현재의']


In [7]:
# 문장, 후보 키워드 임베딩(KoBERT활용)

doc_embeder = SentenceEmbeder(512)
doc_embedding = doc_embeder.encode([document_list[0]])
can_embeder = SentenceEmbeder(5)
candicates_embedding = can_embeder.encode(candidates)


using cached model
using cached model
using cached model
100%|██████████| 1/1 [00:00<00:00,  4.28it/s]
using cached model
using cached model
using cached model
100%|██████████| 35/35 [00:00<00:00, 63.59it/s]


In [8]:
from sklearn.metrics.pairwise import cosine_similarity

top_n = 5
doc_embedding_ = [[elem_elem.item() for elem_elem in elem ]for elem in doc_embedding]
candicates_embedding_ = [[elem_elem.item() for elem_elem in elem ]for elem in candicates_embedding]
print(doc_embedding_)
distances = cosine_similarity(doc_embedding_, candicates_embedding_)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

[[-0.040668562054634094, 0.043865304440259933, -0.29063013195991516, 0.013455611653625965, -0.9887394309043884, 0.9988253712654114, 0.6709423065185547, -0.052721958607435226, 0.0003146152012050152, 0.0032902508974075317, -0.9144020080566406, 0.0015912784729152918, -0.055256664752960205, -0.05453317612409592, 0.0351412259042263, 0.8507722616195679, -0.998702883720398, -0.028378698974847794, 0.05650186166167259, 0.03255175054073334, -0.009855031035840511, -0.05917925387620926, -0.012033941224217415, 0.8820626735687256, -0.014307182282209396, 0.9096845388412476, -0.9936784505844116, 0.044811006635427475, -0.9990421533584595, -0.04372946172952652, -0.7262765169143677, 0.9753170609474182, 0.01700713485479355, -0.018073678016662598, -0.5526136755943298, 0.7137256860733032, -0.02048160880804062, -0.011504387483000755, -0.984975278377533, -0.019496172666549683, 0.022391069680452347, -0.02741526998579502, 0.03510094806551933, 0.9968211054801941, -0.999998927116394, 0.04240431636571884, 0.822438

In [9]:
keywords

['이라고', '보조경기장', '이틀', '발표하면서', '북한의']